In [ ]:
from datetime import datetime
import optuna
import itertools
from collections import Counter
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation

import numpy as np

from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

Data

In [ ]:
train_dir = '../input/fer2013clean/fer2013-clean/Training/'
test_dir = '../input/fer2013clean/fer2013-clean/PublicTest/'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip = True,
                                   zoom_range= [0.8,  1.2],
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                  # rotation_range = 10
                                  # brightness_range = [0.5,1]
                                  ) #values lower than 1 darken the images
test_datagen  = ImageDataGenerator(rescale=1./255)

Data Augmentation

In [ ]:
class MixupImageDataGenerator():
    def __init__(self, generator, directory, batch_size, img_height, img_width, alpha=0.2, subset=None):
        """Constructor for mixup image data generator.
        Arguments:
            generator {object} -- An instance of Keras ImageDataGenerator.
            directory {str} -- Image directory.
            batch_size {int} -- Batch size.
            img_height {int} -- Image height in pixels.
            img_width {int} -- Image width in pixels.
        Keyword Arguments:
            alpha {float} -- Mixup beta distribution alpha parameter. (default: {0.2})
            subset {str} -- 'training' or 'validation' if validation_split is specified in
            `generator` (ImageDataGenerator).(default: {None})
        """

        self.batch_index = 0
        self.batch_size = batch_size
        self.alpha = alpha

        # First iterator yielding tuples of (x, y)
        self.generator1 = generator.flow_from_directory(directory,
                                                        target_size=(img_height, img_width),
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        color_mode='rgb',
                                                        shuffle=True,
                                                        subset=subset)

        # Second iterator yielding tuples of (x, y)
        self.generator2 = generator.flow_from_directory(directory,
                                                        target_size=(img_height, img_width),
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        color_mode='rgb',
                                                        shuffle=True,
                                                        subset=subset)

        # Number of images across all classes in image directory.
        self.n = self.generator1.samples

    def reset_index(self):
        """Reset the generator indexes array.
        """

        self.generator1._set_index_array()
        self.generator2._set_index_array()

    def on_epoch_end(self):
        self.reset_index()

    def reset(self):
        self.batch_index = 0

    def __len__(self):
        # round up
        return (self.n + self.batch_size - 1) // self.batch_size

    def get_steps_per_epoch(self):
        """Get number of steps per epoch based on batch size and
        number of images.
        Returns:
            int -- steps per epoch.
        """

        return self.n // self.batch_size

    def __next__(self):
        """Get next batch input/output pair.
        Returns:
            tuple -- batch of input/output pair, (inputs, outputs).
        """

        if self.batch_index == 0:
            self.reset_index()

        current_index = (self.batch_index * self.batch_size) % self.n
        if self.n > current_index + self.batch_size:
            self.batch_index += 1
        else:
            self.batch_index = 0


        # Get a pair of inputs and outputs from two iterators.
        X1, y1 = self.generator1.next()
        X2, y2 = self.generator2.next()

        # random sample the lambda value from beta distribution.
        b = X1.shape[0]
        l = np.random.beta(self.alpha, self.alpha, b)
        X_l = l.reshape(b, 1, 1, 1)
        y_l = l.reshape(b, 1)

        # Perform the mixup.
        X = X1 * X_l + X2 * (1 - X_l)
        y = y1 * y_l + y2 * (1 - y_l)
        
        return X1, y1

    def __iter__(self):
        while True:
            yield next(self)

Create Training and Validation Generator

In [ ]:
train_generator = MixupImageDataGenerator(generator=train_datagen,
                                          directory=train_dir,
                                          batch_size=64,
                                          img_height=48,
                                          img_width=48)
                                          #color_mode='rgb',
                                          #class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(48,48),
                                                  class_mode='categorical',
                                                  batch_size=1,
                                                  color_mode='rgb',
                                                  shuffle=False)

# setting optuna

In [ ]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_test =test_generator.n//test_generator.batch_size
step_size_test

In [ ]:
dropout_rate = 0.25
lr = 0.0001
penalty = 0.0001
num_classes = 7

#define the CNN model
def create_model(num_layer, mid_units, num_filters,dropout_rate):
    
    model = Sequential()
    model.add(Conv2D(filters=num_filters[0], kernel_size=(3, 3),
                 activation="relu",
                 input_shape=(48, 48, 3)))
    for i in range(1,num_layer):
        model.add(Conv2D(filters=num_filters[i], kernel_size=(3,3), padding="same", activation="relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(mid_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [ ]:
def objective(trial):
    print("Optimize Start")
    
    #clear_session
    keras.backend.clear_session()
    
    #number of the convolution layer
    num_layer = trial.suggest_int("num_layer", 2, 5)
    
    #number of the unit
    mid_units = int(trial.suggest_discrete_uniform("mid_units", 100, 300, 100))
    
    #number of the each convolution layer filter
    num_filters = [int(trial.suggest_discrete_uniform("num_filter_"+str(i), 16, 128, 16)) for i in range(num_layer)]
    
    #dropout_rate
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
    
    model = create_model(num_layer, mid_units, num_filters,dropout_rate)
    model.compile(optimizer=Adam(lr=0.0001), loss="categorical_crossentropy",metrics=["accuracy"])
    
    history = model.fit_generator(train_generator,
                                  steps_per_epoch=train_generator.get_steps_per_epoch(),
                                  epochs = 40, 
                                  verbose=1,
                                  validation_data = test_generator, 
                                  validation_steps = 3586)

    
    scores = model.evaluate(test_generator, steps=step_size_test)
    print('accuracy={}'.format(*scores))
    

    return 1 - history.history["val_accuracy"][-1]

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

In [ ]:
#Function to check optimized parameters

study.best_params

In [ ]:
#Function to check the evaluation value after optimization

study.best_value

In [ ]:
study.trials